## Libraries

In [6]:
import pandas as pd
import numpy as np
import scipy.stats as stat

from math import sqrt
from mlgear.utils import show, display_columns
from surveyweights import normalize_weights, run_weighting_iteration


def margin_of_error(n=None, sd=None, p=None, type='proportion', interval_size=0.95):
    z_lookup = {0.8: 1.28, 0.85: 1.44, 0.9: 1.65, 0.95: 1.96, 0.99: 2.58}
    if interval_size not in z_lookup.keys():
        raise ValueError('{} not a valid `interval_size` - must be {}'.format(interval_size,
                                                                              ', '.join(list(z_lookup.keys()))))
    if type == 'proportion':
        se = sqrt(p * (1 - p)) / sqrt(n)
    elif type == 'continuous':
        se = sd / sqrt(n)
    else:
        raise ValueError('{} not a valid `type` - must be proportion or continuous')
    
    z = z_lookup[interval_size]
    return se * z


def print_pct(pct, digits=0):
    pct = pct * 100
    pct = np.round(pct, digits)
    if pct >= 100:
        if digits == 0:
            val = '>99.0%'
        else:
            val = '>99.'
            for d in range(digits - 1):
                val += '9'
            val += '9%'
    elif pct <= 0:
        if digits == 0:
            val = '<0.1%'
        else:
            val = '<0.'
            for d in range(digits - 1):
                val += '0'
            val += '1%'
    else:
        val = '{}%'.format(pct)
    return val


def calc_result(for_vote, against_vote, n, state=None, interval=0.8):
    GENERAL_POLLING_ERROR = 4.0
    ACQUIESENCE_BIAS = -5.0
    N_SIMS = 1000000
    
    margin_needed_for_victory = 0.5
    if state == 'Florida':
        margin_needed_for_victory = 0.6
    elif state == 'Colorado':
        margin_needed_for_victory = 0.55
    
    for_moe = margin_of_error(n=n, p=for_vote/100, interval_size=interval)
    against_moe = margin_of_error(n=n, p=against_vote/100, interval_size=interval)
    undecided = 100 - for_vote - against_vote
    mean = for_vote + undecided * 0.25 + ACQUIESENCE_BIAS
    raw_moe = for_moe * 100 + against_moe * 100
    
    allocate_undecided = undecided * 0.4
    margin = raw_moe + allocate_undecided + GENERAL_POLLING_ERROR
    
    cdf_value = 0.5 + 0.5 * interval
    normed_sigma = stat.norm.ppf(cdf_value)
    sigma = margin / 100 / normed_sigma
    
    sims = np.random.normal(mean / 100, sigma, N_SIMS)
    chance_pass = np.sum([s > margin_needed_for_victory for s in sims]) / N_SIMS
    low, high = np.percentile(sims, [20, 80]) * 100
    
    return {'mean': mean, 'high': high, 'low': low, 'n': n,
            'raw_moe': raw_moe, 'margin': margin, 'sigma': sigma, 'chance_pass': chance_pass}


def print_result(mean, high, low, n, raw_moe, margin, sigma, chance_pass):
    mean = np.round(mean, 1)
    first = np.round(high, 1)
    second = np.round(low, 1)
    sigma = np.round(sigma * 100, 1)
    raw_moe = np.round(raw_moe, 1)
    margin = np.round(margin, 1)
    chance_pass = print_pct(chance_pass, 1)
    if second < first:
        _ = first
        first = second
        second = _
    if second > 100:
        second = 100
    if first < 0:
        first = 0
    print(('Result {} (80% CI: {} to {}) (N={}) (raw_moe={}pts, margin={}pts, '
           'sigma={}pts) ({} likely to pass)').format(mean,
                                                      first,
                                                      second,
                                                      n,
                                                      raw_moe,
                                                      margin,
                                                      sigma,
                                                      chance_pass))
    print(('{} (80% CI: {} to {}) ({})').format(mean,
                                                first,
                                                second,
                                                chance_pass))
    print('-')

## Load Processed Data

In [7]:
survey = pd.read_csv('responses_processed_national_weighted.csv').fillna('Not presented')

## Other Propositions

In [9]:
measures = ['vote_measure_primate_rights', 'vote_measure_cage_free', 'vote_measure_psilocybin',
            'vote_measure_rank_choice', 'vote_measure_no_labeling', 'vote_measure_marijuana',
            'vote_measure_alt_protein_funding', 'vote_measure_utah_hunt', 'vote_measure_min_wage']

options = ['Vote for / Support', 'Vote against / Oppose', 'Don’t know / Undecided']


for measure in measures:
    print('## {} ##'.format(measure))    
    survey_ = survey.copy()
    survey_ = survey_.loc[survey_[measure].isin(options)].copy()
    survey_['lv_weight'] = normalize_weights(survey_['lv_weight'])
    survey_['lv_weight_alt'] = normalize_weights(survey_['lv_weight_alt'])
    survey_['lv_weight_2020'] = normalize_weights(survey_['lv_weight_2020'])

    print('# LV Weighted')
    lv_weighted_n = int(np.round(survey_['lv_weight'].apply(lambda w: 1 if w > 1 else w).sum()))
    votes = survey_[measure].value_counts(normalize=True) * survey_.groupby(measure)['lv_weight'].mean() * 100
    votes = votes[options] * (100 / votes[options].sum())
    print(votes)
    print('-')
    print('Raw N: {}'.format(len(survey_)))
    print_result(**calc_result(for_vote=votes['Vote for / Support'],
                               against_vote=votes['Vote against / Oppose'],
                               n=lv_weighted_n))
    
    print('# LV Weighted (Alt, Post-Hoc)')
    lv_weighted_n = int(np.round(survey_['lv_weight_alt'].apply(lambda w: 1 if w > 1 else w).sum()))
    votes = survey_[measure].value_counts(normalize=True) * survey_.groupby(measure)['lv_weight_alt'].mean() * 100
    votes = votes[options] * (100 / votes[options].sum())
    print(votes)
    print_result(**calc_result(for_vote=votes['Vote for / Support'],
                               against_vote=votes['Vote against / Oppose'],
                               n=lv_weighted_n))
    
    print('# LV Weighted (2020 National, Post-Hoc)')
    lv_weighted_n = int(np.round(survey_['lv_weight_2020'].apply(lambda w: 1 if w > 1 else w).sum()))
    votes = survey_[measure].value_counts(normalize=True) * survey_.groupby(measure)['lv_weight_2020'].mean() * 100
    votes = votes[options] * (100 / votes[options].sum())
    print(votes)
    print_result(**calc_result(for_vote=votes['Vote for / Support'],
                               against_vote=votes['Vote against / Oppose'],
                               n=lv_weighted_n))

    print('-')
    print('-')
    print('-')

## vote_measure_primate_rights ##
# LV Weighted
Vote for / Support        37.760546
Vote against / Oppose     40.087809
Don’t know / Undecided    22.151645
dtype: float64
-
Raw N: 639
Result 38.3 (80% CI: 24.7 to 51.8) (N=257) (raw_moe=7.8pts, margin=20.6pts, sigma=16.1pts) (23.3% likely to pass)
38.3 (80% CI: 24.7 to 51.8) (23.3%)
-
# LV Weighted (Alt, Post-Hoc)
Vote for / Support        33.133366
Vote against / Oppose     40.798090
Don’t know / Undecided    26.068544
dtype: float64
Result 34.7 (80% CI: 20.0 to 49.3) (N=244) (raw_moe=7.9pts, margin=22.3pts, sigma=17.4pts) (18.9% likely to pass)
34.7 (80% CI: 20.0 to 49.3) (18.9%)
-
# LV Weighted (2020 National, Post-Hoc)
Vote for / Support        37.300060
Vote against / Oppose     40.826138
Don’t know / Undecided    21.873801
dtype: float64
Result 37.8 (80% CI: 24.2 to 51.3) (N=253) (raw_moe=7.8pts, margin=20.6pts, sigma=16.1pts) (22.3% likely to pass)
37.8 (80% CI: 24.2 to 51.3) (22.3%)
-
-
-
-
## vote_measure_cage_free ##
# LV Weig

Result 68.3 (80% CI: 59.6 to 77.1) (N=279) (raw_moe=6.6pts, margin=13.3pts, sigma=10.4pts) (96.1% likely to pass)
68.3 (80% CI: 59.6 to 77.1) (96.1%)
-
-
-
-


In [ ]:
"""
POTUS_CENSUS = {'Alabama': {'Hillary Clinton': 0.3436, 'Donald Trump': 0.6208},
                'Alaska': {'Hillary Clinton': 0.3655, 'Donald Trump': 0.5128},
                'Arizona': {'Hillary Clinton': 0.4513, 'Donald Trump': 0.4867},
                'Arkansas': {'Hillary Clinton': 0.3365, 'Donald Trump': 0.6057},
                'California': {'Hillary Clinton': 0.6173, 'Donald Trump': 0.3162},
                'Colorado': {'Hillary Clinton': 0.4816, 'Donald Trump': 0.4325},
                'Connecticut': {'Hillary Clinton': 0.5457, 'Donald Trump': 0.4093},
                'Delaware': {'Hillary Clinton': 0.531, 'Donald Trump': 0.417},
                'Washington DC': {'Hillary Clinton': 0.905, 'Donald Trump': 0.016},
                'Florida': {'Hillary Clinton': 0.478, 'Donald Trump': 0.490},
                'Georgia': {'Hillary Clinton': 0.456, 'Donald Trump': 0.508},
                'Hawaii': {'Hillary Clinton': 0.622, 'Donald Trump': 0.300},
                'Idaho': {'Hillary Clinton': 0.275, 'Donald Trump': 0.593},
                'Illinois': {'Hillary Clinton': 0.558, 'Donald Trump': 0.379},
                'Indiana': {'Hillary Clinton': 0.379, 'Donald Trump': 0.511},
                'Iowa': {'Hillary Clinton': 0.417, 'Donald Trump': 0.512},
                'Kansas': {'Hillary Clinton': 0.361, 'Donald Trump': 0.567},
                'Kentucky': {'Hillary Clinton': 0.327, 'Donald Trump': 0.625},
                'Louisiana': {'Hillary Clinton': 0.385, 'Donald Trump': 0.581},
                'Maine': {'Hillary Clinton': 0.478, 'Donald Trump': 0.449},
                'Maryland': {'Hillary Clinton': 0.603, 'Donald Trump': 0.339},
                'Massachusetts': {'Hillary Clinton': 0.600, 'Donald Trump': 0.328},
                'Michigan': {'Hillary Clinton': 0.473, 'Donald Trump': 0.475},
                'Minnesota': {'Hillary Clinton': 0.464, 'Donald Trump': 0.449},
                'Mississippi': {'Hillary Clinton': 0.401, 'Donald Trump': 0.579},
                'Missouri': {'Hillary Clinton': 0.401, 'Donald Trump': 0.579},
                'Montana': {'Hillary Clinton': 0.381, 'Donald Trump': 0.562},
                'Nebraska': {'Hillary Clinton': 0.337, 'Donald Trump': 0.588},
                'Nevada': {'Hillary Clinton': 0.479, 'Donald Trump': 0.455},
                'New Hampshire': {'Hillary Clinton': 0.470, 'Donald Trump': 0.466},
                'New Jersey': {'Hillary Clinton': 0.555, 'Donald Trump': 0.414},
                'New Mexico': {'Hillary Clinton': 0.483, 'Donald Trump': 0.404},
                'New York': {'Hillary Clinton': 0.590, 'Donald Trump': 0.365},
                'North Carolina': {'Hillary Clinton': 0.462, 'Donald Trump': 0.498},
                'North Dakota': {'Hillary Clinton': 0.272, 'Donald Trump': 0.630},
                'Ohio': {'Hillary Clinton': 0.436, 'Donald Trump': 0.517},
                'Oklahoma': {'Hillary Clinton': 0.289, 'Donald Trump': 0.653},
                'Oregon': {'Hillary Clinton': 0.501, 'Donald Trump': 0.391},
                'Pennsylvania': {'Hillary Clinton': 0.475, 'Donald Trump': 0.481},
                'Rhode Island': {'Hillary Clinton': 0.544, 'Donald Trump': 0.389},
                'South Carolina': {'Hillary Clinton': 0.407, 'Donald Trump': 0.549},
                'South Dakota': {'Hillary Clinton': 0.317, 'Donald Trump': 0.615},
                'Tennessee': {'Hillary Clinton': 0.347, 'Donald Trump': 0.607},
                'Texas': {'Hillary Clinton': 0.432, 'Donald Trump': 0.522},
                'Utah': {'Hillary Clinton': 0.275, 'Donald Trump': 0.454},
                'Vermont': {'Hillary Clinton': 0.567, 'Donald Trump': 0.303},
                'Virginia': {'Hillary Clinton': 0.497, 'Donald Trump': 0.444},
                'Washington': {'Hillary Clinton': 0.525, 'Donald Trump': 0.368},
                'West Virginia': {'Hillary Clinton': 0.264, 'Donald Trump': 0.685},
                'Wisconsin': {'Hillary Clinton': 0.465, 'Donald Trump': 0.472},
                'Wyoming': {'Hillary Clinton': 0.216, 'Donald Trump': 0.674 }}

MEASURES_STATES = {'Oregon Psilocybin Program Initiative': {'var': 'vote_measure_psilocybin', 'state': 'Oregon'},
                   'DC Initiative 81 (Psilocybin)':  {'var': 'vote_measure_psilocybin', 'state': 'Washington DC'},
                   'Colorado Gray Wolf Reintroduction Initiative': {'var': 'vote_measure_co_gray_wolf', 'state': 'Colorado'},
                   'Montana Firearms': {'var': 'vote_measure_mt_firearms', 'state': 'Montana'},
                   'Alabama Amendment 1 (Citizen Voting)': {'var': 'vote_measure_al_citizen_voting', 'state': 'Alabama'},
                   'Utah Right to Hunt and Fish Amendment': {'var': 'vote_measure_utah_hunt', 'state': 'Utah'},
                   'Louisiana Amendment 1 (No Right to Abortion)': {'var': 'vote_measure_la_a1_abortion', 'state': 'Louisiana'},
                   'Michigan Proposal 2 (Search Warrant for Electronic Data Amendment)': {'var': 'vote_measure_mi_prop2_electronic_privacy', 'state': 'Michigan'},
                   'Nebraska Initiative 428 (Payday Lender Interest Rate Cap Initiative)': {'var': 'vote_measure_ne428_payday_cap', 'state': 'Nebraska'},
                   'Nevada Question 6 (Renewable Energy Standards Initiative)': {'var': 'vote_measure_nvq6_renewables_mandate', 'state': 'Nevada'},
                   'Utah Constitutional Amendment C (Remove Slavery as Punishment for a Crime from Constitution)': {'var': 'vote_measure_no_slavery', 'state': 'Utah'},
                   'Massachusetts Question 2 (Ranked Choice Voting)': {'var': 'vote_measure_rank_choice', 'state': 'Massachusetts'},
                   'Arizona Proposition 207 (Legalize Marijiuna)': {'var': 'vote_measure_marijuana', 'state': 'Arizona'},
                   'Alabama Amendment 5 (Stand Your Ground)': {'var': 'vote_measure_stand_ground', 'state': 'Alabama'},
                   'Arizona Proposition 208 (Income Tax for Education)': {'var': 'vote_measure_income_tax_for_educ', 'state': 'Arizona'},
                   'Colorado Proposition 115 (22-Week Abortion Ban Initiative)': {'var': 'vote_measure_22wk_abortion', 'state': 'Colorado'},
                   'Colorado Proposition 118 (Paid Medical and Family Leave Initiative)': {'var': 'vote_measure_family_leave', 'state': 'Colorado'},
                   'Colorado Proposition EE (Tobacco and E-Cigarette Tax Increase for Health and Education Programs Measure)': {'var': 'vote_measure_cig_tax', 'state': 'Colorado'},
                   'Florida Amendment 2 ($15 Minimum Wage Initiative)': {'var': 'vote_measure_min_wage', 'state': 'Florida'}}

options = ['Vote for / Support', 'Vote against / Oppose', 'Don’t know / Undecided']


for measure_name, measure_data in MEASURES_STATES.items():
    print('## {} ##'.format(measure_name))
    state = measure_data['state']
    measure = measure_data['var']
    
    state_survey = survey.copy()
    potus_census = {'vote2016': POTUS_CENSUS[state].copy()}
    potus_census['vote2016']['Other'] = 1 - potus_census['vote2016']['Hillary Clinton'] - potus_census['vote2016']['Donald Trump']
    output = run_weighting_iteration(state_survey, census=potus_census, weigh_on=['vote2016'], verbose=0)
    potus_weights = output['weights']['vote2016']
    potus_weights = state_survey['vote2016'].astype(str).replace(potus_weights)
    state_survey['weight'] = normalize_weights(state_survey['weight'] * potus_weights)
    state_survey['lv_weight'] = normalize_weights(state_survey['weight'] * state_survey['lv_index'])

    state_survey_ = state_survey.loc[state_survey[measure].isin(options)].copy()
    state_survey_['weight'] = normalize_weights(state_survey_['weight'])
    state_survey_['rv_weight'] = normalize_weights(state_survey_['rv_weight'])
    state_survey_['lv_weight'] = normalize_weights(state_survey_['lv_weight'])

    lv_weighted_n = int(np.round(state_survey_['lv_weight'].apply(lambda w: 1 if w > 1 else w).sum()))
    votes = state_survey_[measure].value_counts(normalize=True) * state_survey_.groupby(measure)['lv_weight'].mean() * 100
    votes = votes[options] * (100 / votes[options].sum())
    print(votes)
    print('Raw N: {}'.format(len(state_survey_)))
    print_result(**calc_result(for_vote=votes['Vote for / Support'],
                               against_vote=votes['Vote against / Oppose'],
                               state=state,
                               n=lv_weighted_n))
    print('-')
"""